<a href="https://colab.research.google.com/github/mystakhs/test-Self-Supervised-Learning/blob/main/VideoMAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VideoMAEの動作確認
目的：固定カメラ動画から 組み立て作業クラス を自動取得できるか 短時間 GPU で検証
戦略：

事前学習済み VideoMAE-Base（または公開が出次第 V-JEPA Tiny）を 特徴抽出器として凍結

数十本だけ手ラベルしたクリップで 線形ヘッド を微調整 (≒ 30 min on T4)
## 2-1. Google Colab 環境セットアップ

In [ ]:
# Colab 起動セル
!pip install -q transformers==4.40 timm decord einops accelerate datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 2-2. 最小サンプルデータ

In [ ]:
from datasets import load_dataset

# Tiny-Kinetics-400から5クラスだけ取得
ds = load_dataset("datalab/Tiny-Kinetics-400", split="train[:50]")

# クラス名を確認して、適当な2クラスだけ使ってもよい
ds = ds.filter(lambda x: x["label"] in [10, 20])  # 適宜クラスID調整


In [ ]:
from datasets import load_dataset
# UCF101 から 2 クラス (e.g., 'PushUps', 'JumpingJack') だけ取得し 10 本ずつ
# ds = load_dataset("ucf101", "bringToOther", split="train[:20]")
ds = load_dataset("flwrlabs/ucf1011", split="train[:20]")

# 独自工場動画は drive に mp4 を置き、load_dataset で VideoFolder 形式でも OK


## 2-3. モデル & 前処理

In [ ]:
from transformers import VideoMAEImageProcessor, VideoMAEForVideoClassification
import torch, torch.nn as nn

id_pretrain = "MCG-NJU/videomae-base-finetuned-kinetics"  # HF ckpt :contentReference[oaicite:6]{index=6}
processor = VideoMAEImageProcessor.from_pretrained(id_pretrain)
model = VideoMAEForVideoClassification.from_pretrained(id_pretrain)

# ↓ 軽量 PoC: エンコーダ凍結し線形層だけ再定義
for p in model.parameters():
    p.requires_grad = False
num_cls = 2                      # ← 作業ステップ数に合わせる
model.classifier = nn.Linear(model.config.hidden_size, num_cls)


## 2-4. データローダ（16 frames, 112² px で省メモリ）

In [ ]:
import decord, torchvision.transforms as T
decord.bridge.set_bridge('torch')

tube_size = 16
img_tf = T.Compose([
    T.Resize(128),
    T.CenterCrop(112),
    T.Normalize([0.45], [0.225])
])

def collate(batch):
    videos = [b["video"][:tube_size].permute(0,3,1,2) / 255. for b in batch]
    videos = torch.stack([img_tf(v) for v in videos])            # [B, T, C, H, W]
    labels = torch.tensor([b["label"] for b in batch])
    return {"pixel_values": videos, "labels": labels}

## 2-5. 15 分で線形プローブ

In [ ]:
from transformers import Trainer, TrainingArguments
args = TrainingArguments(
    "videomae_poc",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    fp16=True,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="no"
)
trainer = Trainer(model, args, train_dataset=ds, eval_dataset=ds, data_collator=collate)
trainer.train()


メモリ目安: T4 GPU (15 GB) で < 5 GB 使用。学習は ≈ 2 it/s → 3 epoch で 10 min 程度。

## 2-6. 予測 & 作業時間計測

In [ ]:
import numpy as np, cv2
def predict_clip(path):
    vr = decord.VideoReader(path, num_threads=1)
    frames = torch.tensor(vr.get_batch(range(0,len(vr), max(1, len(vr)//16))))
    inputs = processor(frames.permute(0,3,1,2), return_tensors="pt")
    logits = model(**inputs).logits
    return logits.softmax(-1).argmax(-1).item()

# ストリーム全体を 2 秒ごとに切り出し → 連続同ラベル区間を統計して作業時間に変換


## 2-7. V-JEPA を試す場合

In [ ]:
!pip install -q jinaai-jeparch  # *仮*: 近い公開実装
from jepa import VJEPAEncoder        # Tiny weight (128 × 16) を使用
# エンコーダ出力を上記線形ヘッドに差し替えるだけ


V-JEPA は マスク不要×高速 なので凍結特徴抽出なら VideoMAE と同コスト。公開 weight が少ないため、まず VideoMAE Tiny で PoC→将来置換が現実的。